In [5]:
%%writefile assignment43.cu
// Записываем код  в файл для компиляции CUDA
// Assignment 4 Task 3
// Реализовать гибридную программу
// Первую часть массива обработайте на CPU, вторую — на GPU
// Сравнить время выполнения

#include <iostream>               // Для стандартный ввод-вывод
#include <cuda_runtime.h>         // Для CUDA Runtime API
#include <chrono>                 // Для измерения времени

using namespace std;              // Чтобы не писать std::

// CUDA KERNEL
// CUDA-ядро для суммирования элементов массива
__global__ void sumKernel(int *d_array, int *d_result, int n) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;       // Вычисляем глобальный индекс потока
    if (idx < n)                                           // Проверка выхода за границы массива
        atomicAdd(d_result, d_array[idx]);                 // Атомарное добавление элемента к результату
}

// Основная функция
int main() {
    const int N = 1000000;                                // Размер массива
    const int SIZE = N * sizeof(int);                     // Размер массива в байтах

    int *h_array = new int[N];                             // Выделяем память на CPU для массива
    int h_result_cpu = 0;                                  // Результат для CPU
    int h_result_gpu = 0;                                  // Результат для GPU
    int h_result_hybrid = 0;                               // Результат гибридного подхода

    // Инициализация массива
    for (int i = 0; i < N; i++)
        h_array[i] = 1;                                    // Заполняем единицами

    // CPU сумма
    auto start_cpu = chrono::high_resolution_clock::now(); // Старт таймера CPU
    for (int i = 0; i < N; i++)                            // Последовательная сумма всего массива
        h_result_cpu += h_array[i];
    auto end_cpu = chrono::high_resolution_clock::now();   // Конец таймера CPU
    chrono::duration<double> cpu_time = end_cpu - start_cpu;// Вычисляем продолжительность времени

    //  GPU сумма
    int *d_array, *d_result;                                // Указатели на GPU
    cudaMalloc(&d_array, SIZE);                             // Выделяем память GPU под массив
    cudaMalloc(&d_result, sizeof(int));                     // Выделяем память GPU под результат
    cudaMemcpy(d_array, h_array, SIZE, cudaMemcpyHostToDevice); // Копируем массив CPU → GPU
    cudaMemset(d_result, 0, sizeof(int));                   // Обнуляем результат на GPU

    int threadsPerBlock = 256;                                       // Потоки в блоке
    int blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock; // Количество блоков

    cudaEvent_t start_gpu, stop_gpu;                                // Создаём события CUDA для измерения времени GPU
    cudaEventCreate(&start_gpu);                                    // Инициализация события старта
    cudaEventCreate(&stop_gpu);                                     // Инициализация события конца
    cudaEventRecord(start_gpu);                                     // Запускаем событие старта

    sumKernel<<<blocksPerGrid, threadsPerBlock>>>(d_array, d_result, N); // Запуск ядра GPU

    cudaEventRecord(stop_gpu);                                      // Отмечаем событие конца
    cudaEventSynchronize(stop_gpu);                                 // Ждём завершения ядра GPU
    float gpu_time = 0;                                             // Переменная для хранения времени GPU
    cudaEventElapsedTime(&gpu_time, start_gpu, stop_gpu);           // Время GPU в миллисекундах

    cudaMemcpy(&h_result_gpu, d_result, sizeof(int), cudaMemcpyDeviceToHost); // Копируем результат GPU → CPU

    // Сумма Гибрид
    int half = N / 2;                                      // Делим массив пополам
    int h_result_cpu_part = 0;                             // Результат CPU половины массива

    auto start_hybrid = chrono::high_resolution_clock::now(); // Старт таймера гибрида

    // CPU обрабатывает первую половину массива
    for (int i = 0; i < half; i++)                            // Проходим по первой половине массива
        h_result_cpu_part += h_array[i];                      // Суммируем элементы в h_result_cpu_part

    // GPU обрабатывает вторую половину массива
    int gpu_half_size = N - half;                             // Размер второй половины
    int *d_array_half, *d_result_half;                        // Указатели для второй половины на GPU
    cudaMalloc(&d_array_half, gpu_half_size * sizeof(int));   // Память GPU под вторую половину
    cudaMalloc(&d_result_half, sizeof(int));                  // Результат GPU
    cudaMemcpy(d_array_half, h_array + half, gpu_half_size * sizeof(int), cudaMemcpyHostToDevice); // Копируем вторую половину
    cudaMemset(d_result_half, 0, sizeof(int));                // Обнуляем результат

    int blocks_hybrid = (gpu_half_size + threadsPerBlock - 1) / threadsPerBlock;                // Количество блоков для второй половины
    sumKernel<<<blocks_hybrid, threadsPerBlock>>>(d_array_half, d_result_half, gpu_half_size); // Запуск ядра GPU

    int h_result_gpu_part = 0;                                                          // Переменная для хранения результата GPU половины
    cudaMemcpy(&h_result_gpu_part, d_result_half, sizeof(int), cudaMemcpyDeviceToHost); // Копируем результат GPU половины

    h_result_hybrid = h_result_cpu_part + h_result_gpu_part; // Складываем результаты CPU и GPU

    auto end_hybrid = chrono::high_resolution_clock::now();  // Конец таймера гибрида
    chrono::duration<double> hybrid_time = end_hybrid - start_hybrid;  // Вычисляем продолжительность времени

    // Вывод результатов
    cout << "Результат CPU: " << h_result_cpu << endl;             // Выводим результат CPU
    cout << "Результат GPU: " << h_result_gpu << endl;             // Выводим результат GPU
    cout << "Результат гибрида: " << h_result_hybrid << endl;      // Выводим результат гибридного метода

    cout << "Время CPU: " << cpu_time.count() * 1000 << " мс" << endl;
    cout << "Время GPU: " << gpu_time << " мс" << endl;
    cout << "Время гибрида: " << hybrid_time.count() * 1000 << " мс" << endl;

    // Вычисляем ускорение
    double speedup_gpu = (cpu_time.count() * 1000) / gpu_time;                        // На сколько раз GPU быстрее CPU
    double speedup_hybrid = (cpu_time.count() * 1000) / (hybrid_time.count() * 1000); // На сколько раз гибрид быстрее CPU
    cout << "GPU работает примерно в " << speedup_gpu << " раз быстрее CPU." << endl;
    cout << "Гибрид работает примерно в " << speedup_hybrid << " раз быстрее CPU." << endl;

    // Освобождение памяти
    delete[] h_array;                                       // Освобождаем CPU память
    cudaFree(d_array);                                      // Освобождаем память GPU
    cudaFree(d_result);
    cudaFree(d_array_half);
    cudaFree(d_result_half);

    return 0;                                               // Завершаем программу
}


Overwriting assignment43.cu


In [6]:
# Компиляция
!nvcc assignment43.cu -o assignment43 -arch=sm_75 -std=c++11            # -arch=sm_75  - архитектура GPU (Tesla T4 в Colab = sm_75)
                                                                        # -std=c++11 — стандарт C++
# Запуск
!./assignment43

Результат CPU: 1000000
Результат GPU: 1000000
Результат гибрида: 1000000
Время CPU: 2.35736 мс
Время GPU: 0.135296 мс
Время гибрида: 1.92502 мс
GPU работает примерно в 17.4237 раз быстрее CPU.
Гибрид работает примерно в 1.22459 раз быстрее CPU.
